In [1]:
import os
import pandas as pd
import numpy as np
import scipy
import datetime
from sklearn.model_selection import train_test_split
import gc, time
#os.chdir('C:/Users/Paperspace/Desktop/Kaggle-Competitions/FraudDetection')
os.chdir('C:\Users/dhruv/Desktop/Kaggle/TalkingData/data')

In [2]:
dtypes = {
    'ip'            : 'uint32',
    'app'           : 'uint16',
    'device'        : 'uint16',
    'os'            : 'uint16',
    'channel'       : 'uint16',
    'is_attributed' : 'uint8',
    'click_id'      : 'uint32'
}

In [3]:
def handleClicktime(df):
    df['click_hour']= (pd.to_datetime(df['click_time']).dt.round('H')).dt.hour
    df['click_hour'] = df['click_hour'].astype('uint16')
    df = df.drop(['click_time'], axis=1)   
    return df

In [78]:
st_time = time.time()
train = pd.read_csv('train.csv', dtype=dtypes, skiprows=range(1,133333333), nrows=33333333)
print ('Time to load train [{}] sec'.format(time.time()-st_time))

train_record_index = train.shape[0]

Time to load train [115.85464644432068] sec


In [79]:
st_time = time.time()
test = pd.read_csv('test.csv')
print ('Time to load test [{}] sec'.format(time.time()-st_time))

Time to load test [18.50202703475952] sec


In [80]:
# Handle click time
st_time = time.time()
train = handleClicktime(train)
test = handleClicktime(test)
print ('Time to handle click time [{}] sec'.format(time.time()-st_time))
gc.collect()

Time to handle click time [29.993566751480103] sec


359

In [81]:
#df for submit
df_submit = pd.DataFrame()
df_submit['click_id'] = test['click_id']

In [82]:
# Get y
Learning_Y = train['is_attributed']

In [83]:
#drop zone
test = test.drop(['click_id'], axis=1)
train = train.drop(['is_attributed', 'attributed_time'], axis=1)
gc.collect()

35

In [84]:
df_merge = pd.concat([train, test])
del train, test
gc.collect()

0

In [85]:
# Count ip for both train and test df 
start_time = time.time()
df_ip_count = df_merge['ip'].value_counts().reset_index(name='ip_count')
df_ip_count.columns = ['ip', 'ip_count']
print('Load df_ip_count with [{}] seconds'.format(time.time() - start_time))

Load df_ip_count with [5.974506616592407] seconds


In [86]:
df_merge = df_merge.merge(df_ip_count, on='ip', how='left', sort=False)
df_merge['ip_count'] = df_merge['ip_count'].astype('uint16')

In [87]:
df_merge = df_merge.drop(['ip'], axis=1)
del df_ip_count
gc.collect()

42

In [88]:
train = df_merge[:train_record_index]
test = df_merge[train_record_index:]

del df_merge
gc.collect()

0

In [38]:
Learning_Y.head()

0    0
1    0
2    0
3    0
4    0
Name: is_attributed, dtype: uint8

# Light GBM

In [92]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score
import time

In [93]:
def recall_lgb(preds, d_train):
    labels = d_train.get_label()
    preds = np.where((preds > 0.5), 1, 0)
    recs = recall_score(preds, labels)
    return 'recall', recs, True

In [94]:
params = {'boosting' : 'gbdt', 'learning_rate' : 0.02, 'application' : 'binary', 'max_depth' : 7, 'num_leaves' : 26, 
         'data_random_seed' : 42, 'bagging_fraction' : 0.4, 'nthread' : 8, 'metric' : 'auc', 'is_unbalanced' : True}

In [96]:
evals_results = {}
d_train = lgb.Dataset(train, label = Learning_Y)

watchlist = [d_train]

model = lgb.train(params, d_train, valid_sets = watchlist, evals_result = evals_results, num_boost_round = 1000, early_stopping_rounds = 30)

[1]	training's auc: 0.870957
Training until validation scores don't improve for 30 rounds.
[2]	training's auc: 0.870964
[3]	training's auc: 0.870964
[4]	training's auc: 0.870984
[5]	training's auc: 0.870988
[6]	training's auc: 0.870991
[7]	training's auc: 0.870991
[8]	training's auc: 0.870991
[9]	training's auc: 0.871002
[10]	training's auc: 0.871001
[11]	training's auc: 0.871002
[12]	training's auc: 0.871005
[13]	training's auc: 0.871005
[14]	training's auc: 0.871006
[15]	training's auc: 0.871011
[16]	training's auc: 0.871011
[17]	training's auc: 0.871018
[18]	training's auc: 0.903166
[19]	training's auc: 0.90316
[20]	training's auc: 0.903329
[21]	training's auc: 0.903439
[22]	training's auc: 0.903451
[23]	training's auc: 0.90345
[24]	training's auc: 0.903455
[25]	training's auc: 0.903454
[26]	training's auc: 0.903458
[27]	training's auc: 0.90346
[28]	training's auc: 0.903467
[29]	training's auc: 0.903468
[30]	training's auc: 0.903467
[31]	training's auc: 0.903475
[32]	training's auc:

[269]	training's auc: 0.9555
[270]	training's auc: 0.955512
[271]	training's auc: 0.955525
[272]	training's auc: 0.955586
[273]	training's auc: 0.955597
[274]	training's auc: 0.955746
[275]	training's auc: 0.955745
[276]	training's auc: 0.955767
[277]	training's auc: 0.955823
[278]	training's auc: 0.955844
[279]	training's auc: 0.955878
[280]	training's auc: 0.955886
[281]	training's auc: 0.955886
[282]	training's auc: 0.955888
[283]	training's auc: 0.955925
[284]	training's auc: 0.955932
[285]	training's auc: 0.955955
[286]	training's auc: 0.955958
[287]	training's auc: 0.955956
[288]	training's auc: 0.955972
[289]	training's auc: 0.956324
[290]	training's auc: 0.956317
[291]	training's auc: 0.95646
[292]	training's auc: 0.956479
[293]	training's auc: 0.956476
[294]	training's auc: 0.956498
[295]	training's auc: 0.956522
[296]	training's auc: 0.95656
[297]	training's auc: 0.956769
[298]	training's auc: 0.956807
[299]	training's auc: 0.956822
[300]	training's auc: 0.956835
[301]	traini

[535]	training's auc: 0.964533
[536]	training's auc: 0.964549
[537]	training's auc: 0.964557
[538]	training's auc: 0.964581
[539]	training's auc: 0.964601
[540]	training's auc: 0.964611
[541]	training's auc: 0.964627
[542]	training's auc: 0.964657
[543]	training's auc: 0.964671
[544]	training's auc: 0.964697
[545]	training's auc: 0.964699
[546]	training's auc: 0.964706
[547]	training's auc: 0.964725
[548]	training's auc: 0.964733
[549]	training's auc: 0.964757
[550]	training's auc: 0.964781
[551]	training's auc: 0.964784
[552]	training's auc: 0.964789
[553]	training's auc: 0.964801
[554]	training's auc: 0.96483
[555]	training's auc: 0.964841
[556]	training's auc: 0.964844
[557]	training's auc: 0.96485
[558]	training's auc: 0.964875
[559]	training's auc: 0.964878
[560]	training's auc: 0.964881
[561]	training's auc: 0.964887
[562]	training's auc: 0.964913
[563]	training's auc: 0.964927
[564]	training's auc: 0.964954
[565]	training's auc: 0.964956
[566]	training's auc: 0.964979
[567]	trai

KeyboardInterrupt: 

In [97]:
# save model
model.save_model('lgbmmodel.txt')

NameError: name 'model' is not defined

In [ ]:
# predict
y_pred = model.predict(test, num_iteration=model.best_iteration)

In [ ]:
# delete training files
del d_train
del d_valid
del train_X
del valid_X
del train_y
del valid_y

# random forest

In [39]:
from sklearn.ensemble import RandomForestClassifier

#clf = RandomForestClassifier(n_estimators=13, max_depth=13, random_state=44,verbose=2)
#clf.fit(train, Learning_Y)

building tree 1 of 13


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.7min remaining:    0.0s


building tree 2 of 13
building tree 3 of 13
building tree 4 of 13
building tree 5 of 13
building tree 6 of 13
building tree 7 of 13
building tree 8 of 13
building tree 9 of 13
building tree 10 of 13
building tree 11 of 13
building tree 12 of 13
building tree 13 of 13


[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed: 50.9min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=13, n_jobs=1,
            oob_score=False, random_state=44, verbose=2, warm_start=False)

In [89]:
predictions = clf.predict_proba(test)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:   34.7s finished


In [90]:
df_submit['is_attributed'] = predictions[:,1]
df_submit.describe()

,click_id,is_attributed
count,1.879047e+07,1.879047e+07
mean,9.395234e+06,3.671852e-03
std,5.424341e+06,3.458478e-02
min,0.000000e+00,0.000000e+00
25%,4.697617e+06,2.936116e-04
50%,9.395234e+06,4.071790e-04
75%,1.409285e+07,6.634249e-04
max,1.879047e+07,1.000000e+00


In [91]:
df_submit.to_csv('submission.csv', index = False)